# Pre-Processing Method
This method loads in two object detection models and runs them on video data. First the videos are split into frames at a rate of 4fps, the image quality is reduced to 800 pixels, and then they are filtered, removing frames with a greyscale less than 5, and those with an absolute difference of less than 2000. Additionally, at least one frame is taken every 2 seconds, and after a period of no frames being taken becuase they all have an absolute difference less than 2000, the final frame of this period is also taken. 

In [ ]:
import torch
import os
from ultralytics import YOLO
import cv2
import fnmatch
import csv
import numpy as np
from concurrent.futures import ProcessPoolExecutor
import concurrent.futures
from multiprocessing import Manager
import tempfile

In [ ]:
%cd Brands-YOLO8-Model/

In [ ]:
brands_model = YOLO('runs/detect/train/weights/best.pt')

In [ ]:
%cd '~'
%cd 'Desktop'

In [ ]:
%cd Alcohol-YOLO8-Model/

In [ ]:
alc_model = YOLO('runs/detect/ALCOHOL_100/weights/best.pt')

In [ ]:
%cd '~'

In [ ]:
#Output file names
ALC_outputFilename = "Desktop/Frames/Output-Alc.csv" 
if os.path.exists(ALC_outputFilename):
    print(f"Predictions file already exists in the \'{ALC_outputFilename}\'.")
    print(f"Please delete the existing {ALC_outputFilename} to rerun the code.")

BRANDS_outputFilename = "Desktop/Frames/Output-Brands.csv" 
if os.path.exists(BRANDS_outputFilename):
    print(f"Predictions file already exists in the \'{BRANDS_outputFilename}\'.")
    print(f"Please delete the existing {BRANDS_outputFilename} to rerun the code.")

In [ ]:
#check the absolute difference between two frames
def frames_equal(frame1, frame2):
    gray_frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray_frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    diff = cv2.absdiff(gray_frame1, gray_frame2)
    _, diff_threshold = cv2.threshold(diff, 50, 255, cv2.THRESH_BINARY)
    diff_count = cv2.countNonZero(diff_threshold)
    
    return diff_count < 2000

#check the greyscale intesity of a frame
def overall_grayscale_intensity(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    intensity = cv2.mean(gray_frame)[0]
    
    return intensity

In [ ]:
# Seperates individual predictions and retrievals all relevant information for post-processing
def process_results(results):
    ret = []
    conf = results[0].boxes.conf
    conf = conf.detach().cpu().numpy().tolist()
    for r in results:
        res = []
        nums = r.boxes.cls.numpy()
        names = r.names
        for i in nums:
            res.append(names[i])
        print(res)
    if not res:
        return 0
    else: 
        locations = results[0].boxes.xywh
        locations = locations.detach().cpu().numpy().tolist()
        locations = [':'.join(f'{round(num, 2)}' for num in sublist) for sublist in locations]
        for i in range(len(res)):
            ret.append([conf[i], res[i], locations[i]])
    return ret

In [ ]:
def extract_frames(video_path):
    alc_rows = []
    brands_rows = []
    video_capture = cv2.VideoCapture(video_path) #load the video
    
    fps = video_capture.get(cv2.CAP_PROP_FPS)
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT)) #count the total frames
    
    frame_counter = 0
    prev_frame = None  
    last_counter = 0
    last=True
    
    while frame_counter < total_frames: #iterate through the frames
        frame_pos = int(frame_counter)
        video_capture.set(cv2.CAP_PROP_POS_FRAMES, frame_pos) #extract frame at each time point

        ret, frame = video_capture.read()

        if ret:
            height, width = frame.shape[:2]
            if width > height: #resize the frame
                scale_factor = 800 / width
            else:
                scale_factor = 800 / height
            new_width = int(width * scale_factor)
            new_height = int(height * scale_factor)
            frame = cv2.resize(frame, (new_width, new_height))

        if not ret: #break if there is no frame
            break

        if prev_frame is None or not frames_equal(frame, prev_frame) or last_counter>=7: #filters out unessessary frames - keeps the first frame, different frames, and at least one frame every 2 seconds
            if last_counter>=7 and not frames_equal(frame, prev_frame): 
                last = False
            else: 
                if last_counter>=7: last = True
            items = [frame] if last else [frame, prev_frame] 
            timestamp = [frame_counter/fps] if last else [frame_counter/fps, (frame_counter-(fps/4))/fps]
            for item, ts in zip(items, timestamp):
                if overall_grayscale_intensity(item) > 5: #check greyscale
                    minutes, seconds = divmod(ts, 60)
                    hours, minutes = divmod(minutes, 60)
                    time_str = f"{int(hours):02d}_{int(minutes):02d}_{seconds:05.2f}"
                    output_frame_path = f"{os.path.splitext(os.path.basename(video_path))[0]}_{time_str}.jpg"

                    with concurrent.futures.ProcessPoolExecutor(max_workers=2) as executor: #predict each model in parrallel
                        alc_future = executor.submit(process_results, alc_model([item], conf=0.2))
                        brands_future = executor.submit(process_results, brands_model([item], conf=0.2))

                        proc_alc = alc_future.result()
                        proc_brands = brands_future.result()
                    
                    video_path_parts = video_path.split('/')

                        # add the results to a dataframe
                    if proc_alc == 0:
                        print(f"Nothing Here: {output_frame_path}")
                    else: 
                        for i in range(len(proc_alc)):
                            alc_rows.append(f"\n{video_path_parts[1]}, {video_path_parts[2]}, {video_path_parts[3].replace(",", "").replace(" ", "_")}, {video_path_parts[4]},{time_str},{proc_alc[i][0]}, {proc_alc[i][1]}, {proc_alc[i][2]}")
                        print(f"Frame saved: {output_frame_path}")

                    if proc_brands == 0:
                        print(f"Nothing Here: {output_frame_path}")
                    else: 
                        for i in range(len(proc_brands)):
                            brands_rows.append(f"\n{video_path_parts[1]}, {video_path_parts[2]}, {video_path_parts[3].replace(",", "").replace(" ", "_")}, {video_path_parts[4]},{time_str},{proc_brands[i][0]}, {proc_brands[i][1]}, {proc_brands[i][2]}")
                        print(f"Frame saved: {output_frame_path}")
                
                last_counter=0
                last=True
            
        else:
            last_counter = last_counter+1
            last=False
        prev_frame = frame

        frame_counter += fps / 4 # 4fps
    
    video_capture.release()
    print(f"Frames extraction complete for video {video_path}")
    return alc_rows, brands_rows

In [ ]:
def process_video(args): # for each video, call the extract_frame method
    video_path, alc_tmp_file, brands_tmp_file = args
    alc_dataframe, brands_dataframe = extract_frames(video_path)
    
    cleaned_alc_rows = [row.strip() for row in alc_dataframe]
    cleaned_brands_rows = [row.strip() for row in brands_dataframe]

        #append the results for a video to the appropriate file
    if cleaned_alc_rows:
        with open(alc_tmp_file, mode="a") as alc_file:
            alc_file.write("\n".join(cleaned_alc_rows) + "\n")

    if cleaned_brands_rows:
        with open(brands_tmp_file, mode="a") as brands_file:
            brands_file.write("\n".join(cleaned_brands_rows) + "\n")

def extract_videos_from_folder(folder_path, ALC_outputFilename, BRANDS_outputFilename):
    first_videos = []
    for root, dirs, files in os.walk(folder_path): #extract only one video from each end file
        for filename in files:
            if fnmatch.fnmatch(filename, '*.mp4'):
                if "_avo_" not in filename:
                    name = os.path.join(root, filename)
                    if 'analytics' not in name.lower() and 'digital' not in name.lower(): #dont include any analytical videos
                        first_videos.append(name)
                    break
    print(len(first_videos))
    if not first_videos:
        print("No .mp4 files found in any end folder.")
        return

        # create output files
    with open(ALC_outputFilename, mode="w") as alc_file:
        alc_file.write("Student,Device,DateTime,VideoFile,Timestamp,Confidence,Prediction,Location\n")

    with open(BRANDS_outputFilename, mode="w") as brands_file:
        brands_file.write("Student,Device,DateTime,VideoFile,Timestamp,Confidence,Prediction,Location\n")

    with ProcessPoolExecutor(max_workers=15) as executor: #process the videos in parrallel
        executor.map(process_video, [(video, output_path, ALC_outputFilename, BRANDS_outputFilename) for video in first_videos])

In [ ]:
folder_path = "STUDENT_DATA/"
extract_videos_from_folder(folder_path, ALC_outputFilename, BRANDS_outputFilename)